In [3]:
import json
import os
from datetime import datetime
from tqdm import tqdm

def convert_multiple_json_to_coco(input_dir, output_base_dir, image_base_dir):
    # JSON 파일 목록 가져오기
    json_files = [f for f in os.listdir(input_dir) if f.endswith('.json')]

    for json_file in tqdm(json_files, desc="Processing JSON files"):
        # COCO 형식 데이터 초기화
        coco_data = {
            "info": {
                "description": "Converted from custom format to COCO",
                "url": "",
                "version": "1.0",
                "year": datetime.now().year,
                "contributor": "",
                "date_created": datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
            },
            "licenses": [{"url": "", "id": 1, "name": "Unknown"}],
            "images": [],
            "annotations": [],
            "categories": []
        }

        image_id = 1
        ann_id = 1
        category_id_map = {}

        # JSON 파일 열기
        with open(os.path.join(input_dir, json_file), 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 카테고리 추가 또는 가져오기
        species = data['metadata']['species'].lower()
        if species not in category_id_map:
            category_id = len(category_id_map) + 1
            category_id_map[species] = category_id
            coco_data['categories'].append({
                "supercategory": "animal",
                "id": category_id,
                "name": species
            })
        else:
            category_id = category_id_map[species]

        # 이미지 폴더 경로 추출 (JSON 파일명을 기반으로 폴더 찾기)
        json_file_stem = os.path.splitext(json_file)[0]  # 파일 확장자를 제외한 이름
        image_dir = os.path.join(image_base_dir, json_file_stem).replace("\\", "/")  # 경로 수정
        
        print(f"Looking for images in: {image_dir}")  # 디버깅용 출력 추가

        if not os.path.exists(image_dir):
            print(f"Image directory {image_dir} not found. Skipping this JSON.")
            continue

        # 프레임별 이미지 처리
        for annotation in data['annotations']:
            frame_number = annotation['frame_number']
            timestamp = annotation['timestamp']

            # 이미지 파일명 생성
            frame_file = f"frame_{frame_number}_timestamp_{timestamp}.jpg"
            frame_path = os.path.join(image_dir, frame_file).replace("\\", "/")

            # 이미지 파일이 존재하는지 확인
            if os.path.exists(frame_path):
                # 이미지 정보 추가
                image_info = {
                    "id": image_id,
                    "file_name": os.path.join(json_file_stem, frame_file).replace("\\", "/"),
                    "width": data['metadata']['width'],
                    "height": data['metadata']['height'],
                    "license": 1,
                    "flickr_url": "",
                    "coco_url": "",
                    "date_captured": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
                coco_data['images'].append(image_info)

                # 어노테이션 추가
                keypoints = []
                num_keypoints = 0
                for i in range(1, 16):
                    if str(i) in annotation['keypoints'] and annotation['keypoints'][str(i)]:
                        keypoints.extend([
                            annotation['keypoints'][str(i)]['x'],
                            annotation['keypoints'][str(i)]['y'],
                            2  # 2는 가시성을 나타냅니다. 여기서는 모든 키포인트가 보인다고 가정합니다.
                        ])
                        num_keypoints += 1
                    else:
                        keypoints.extend([0, 0, 0])  # 키포인트가 없는 경우

                ann_info = {
                    "id": ann_id,
                    "image_id": image_id,
                    "category_id": category_id,
                    "segmentation": [],  # 세그멘테이션 정보가 없으므로 빈 리스트로 둡니다.
                    "area": annotation['bounding_box']['width'] * annotation['bounding_box']['height'],
                    "bbox": [
                        annotation['bounding_box']['x'],
                        annotation['bounding_box']['y'],
                        annotation['bounding_box']['width'],
                        annotation['bounding_box']['height']
                    ],
                    "iscrowd": 0,
                    "keypoints": keypoints,
                    "num_keypoints": num_keypoints
                }
                coco_data['annotations'].append(ann_info)
                ann_id += 1

                image_id += 1
            else:
                print(f"Image file {frame_path} not found. Skipping this frame.")

        # JSON 파일명에 맞는 COCO 파일 저장 경로 설정
        output_json_path = os.path.join(output_base_dir, f"{json_file_stem}_coco.json").replace("\\", "/")

        # COCO 형식의 JSON 파일 저장
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(coco_data, f, ensure_ascii=False, indent=2)

        print(f"Converted data saved to {output_json_path}")


# 사용 예시
input_dir = 'G:/workspace/test/ARCH'
output_json_path = 'G:/workspace/test/ARCH/coco'
image_dir = 'G:/workspace/test/ARCH/img'

convert_multiple_json_to_coco(input_dir, output_json_path, image_dir)


Processing JSON files:   0%|          | 0/180 [00:00<?, ?it/s]

Processing JSON files:   7%|▋         | 13/180 [00:00<00:01, 108.31it/s]

Looking for images in: G:/workspace/test/ARCH/img/20201028_cat-arch-000156.mp4
Image file G:/workspace/test/ARCH/img/20201028_cat-arch-000156.mp4/frame_3_timestamp_200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/20201028_cat-arch-000156.mp4/frame_6_timestamp_400.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/20201028_cat-arch-000156.mp4/frame_9_timestamp_600.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/20201028_cat-arch-000156.mp4/frame_18_timestamp_1200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/20201028_cat-arch-000156.mp4/frame_21_timestamp_1400.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/20201028_cat-arch-000156.mp4/frame_24_timestamp_1600.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/20201028_cat-arch-000156.mp4/frame_27_timestamp_1800.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/

Processing JSON files:  22%|██▏       | 40/180 [00:00<00:01, 123.48it/s]

Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_276_timestamp_9200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_282_timestamp_9400.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_288_timestamp_9600.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_294_timestamp_9800.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_300_timestamp_10000.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_306_timestamp_10200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_312_timestamp_10400.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_318_timestamp_10600.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-013635/frame_324_timestamp_10

...

Converted data saved to G:/workspace/test/ARCH/coco/cat-arch-049523_coco.json
Looking for images in: G:/workspace/test/ARCH/img/cat-arch-049524
Image file G:/workspace/test/ARCH/img/cat-arch-049524/frame_6_timestamp_200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-049524/frame_246_timestamp_8200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-049524/frame_252_timestamp_8400.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-049524/frame_258_timestamp_8600.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-049524/frame_264_timestamp_8800.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-049524/frame_270_timestamp_9000.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-049524/frame_276_timestamp_9200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-049524/

Processing JSON files:  94%|█████████▍| 170/180 [00:01<00:00, 122.11it/s]

Converted data saved to G:/workspace/test/ARCH/coco/cat-arch-058259_coco.json
Looking for images in: G:/workspace/test/ARCH/img/cat-arch-059581
Image file G:/workspace/test/ARCH/img/cat-arch-059581/frame_6_timestamp_200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-059581/frame_24_timestamp_800.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-059581/frame_30_timestamp_1000.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-059581/frame_36_timestamp_1200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-059581/frame_42_timestamp_1400.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-059581/frame_54_timestamp_1800.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-059581/frame_60_timestamp_2000.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-059581/frame_6

Processing JSON files: 100%|██████████| 180/180 [00:01<00:00, 123.43it/s]

Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_234_timestamp_7800.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_240_timestamp_8000.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_246_timestamp_8200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_252_timestamp_8400.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_258_timestamp_8600.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_264_timestamp_8800.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_270_timestamp_9000.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_276_timestamp_9200.jpg not found. Skipping this frame.
Image file G:/workspace/test/ARCH/img/cat-arch-078783/frame_282_timestamp_9400.j